#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import warnings
warnings.filterwarnings('ignore')
import geopy
from geopy.geocoders import Nominatim
import geocoder
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import HTML, display

#### Set the url to table location and get the content of the page in variable

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).content

#### Use the pandas read_html function to read the table. In this case the first table on the page ([0])

In [3]:
df_raw = pd.read_html(page,header=0)[0]

In [4]:
df_raw.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Let's clean up the dataset according to specifications, filtering and replacing values

In [5]:
df = df_raw[df_raw['Borough'] != 'Not assigned'] #filter
df.columns = ['PostalCode','Borough','Neighborhood'] #set columns
df.loc[df['Neighborhood'] == 'Not assigned',['Neighborhood']] = df['Borough'] #replace Neigborhood with Borough if Neigborhood = 'Not assigned'

#### Group the dataframe and apply the string concatenation

In [6]:
df = df.groupby(by=['PostalCode','Borough']).agg(lambda x: ', '.join(set(x))).reset_index()

#### Finally let's show the final frame (103 records with 3 columns)

In [7]:
df.shape

(103, 3)

#### We have the dataframe setup, now let's find the longitude and latitude. I'm using the provided csv as the geocoder is malfunctioning

In [8]:
file = 'Geospatial_Coordinates.csv'
df_geo = pd.read_csv(file)
df_geo.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)

#### We now have the location data per postalcode so now it will be joined together with the Toronto dataframe

In [9]:
df_toronto = pd.merge(df,df_geo, on='PostalCode',how='left')

#### This leaves with clean appended dataframe incl lat and long

In [10]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Let's start with our first map and plot the locations on the Toronto map

##### We'll define the start zoom location of the folium map with the address and geolocator to get the latitude and longitude

In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


#### Now we plot the areas onto the Toronto map

In [12]:
# Function required to show folium maps inline

In [22]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
map_toronto.save('toronto_map1.html')
display(map_toronto)

#### Now we have the initial map setup we continue with clustering. We will start with limiting the dataset as now we have 103 points on the map. We'll limit the set to only show locations for Borough's with the name Toronto in it

In [ ]:
toronto_data = df_toronto[df_toronto.Borough.str.contains('Toronto')].reset_index(drop=True)
toronto_data.shape

#### This leaves us with 38 Boroughs we wil continue to work with. Let's map them again

In [ ]:
# create map of New York using latitude and longitude values
map_toronto_filtered = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_filtered)  
    
map_toronto_filtered

#### Let's look at venues with foursquare to the locations we have

In [ ]:
# @hidden cell
CLIENT_ID = '5LAMV3DNDHBER3VMUROMJNYRCJ5S35VSIB5BTJKJW2KHVG55' # your Foursquare ID
CLIENT_SECRET = '3D5FLKFOOA41T5XPDDIZTLLWTPIJWAMVQIU3AS5POKUEV1BW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [ ]:
# some variables we need for the below functions

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

#### Function to get venues to process all the neighborhoods in Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#Apply the function
toronto_venues = getNearbyVenues(names=toronto_data ['Neighborhood'],
                                   latitudes=toronto_data ['Latitude'],
                                   longitudes=toronto_data['Longitude'])

#### So we got 1700 records returned with 7 columns

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

#### We continue to analyse the neighborhoods

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

#### So got the categories converted to numerical values and transposed them into columns. We have 1700 records with 234 Venues categories

##### We group them by Neigborhood  and that will leave us with 38 neighborhoods with 234 venue categories

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

#### That is a lot to process so we will get the top 10 venues for each neighborhood

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(3)

#### We will now cluster the neighborhood with *k*-means into 5 clusters

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
kmeans_labels = kmeans.labels_

#### Let's add the clusters back to the neighborhoods and venues

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans_labels)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

##### Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

#### The question then is what do the clusters represent. What is in those various clusters so we can name them better than Cluster 0-4

##### We will filter the toronto_merged frame into their respective variable so we analyse further

In [ ]:
label_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
label_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
label_2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
label_3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
label_4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### I'm not exactly sure how to find the common ground in the various clusters

In [ ]:
venues_columns = neighborhoods_venues_sorted.columns
venues_columns = venues_columns.drop(['Cluster Labels','Neighborhood'])

In [ ]:
label_1.head()

In [ ]:
label_4 # park